## GPTを使用したMoodle小テスト問題（ドラッグ＆ドロップテキスト）の生成

以下のセルのコードを上から順番に実行します。出力になにかおかしなところがあれば、そのセルを再度実行（=regenerate）し、その後のセルも再度実行してください。

Moodleの問題バンクにインポートできるMoodle XMLファイルが作成されます。ドラッグ＆ドロップテキストは、
https://docs.moodle.org/3x/ja/%E3%83%89%E3%83%A9%E3%83%83%E3%82%B0%EF%BC%86%E3%83%89%E3%83%AD%E3%83%83%E3%83%97%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E5%95%8F%E9%A1%8C%E3%82%BF%E3%82%A4%E3%83%97
のような問題形式です。

注1：かなりの確率で正解の指定を間違えるようなので、問題を出題する前にはしっかり確認してください。

注2：比較的複雑な処理をGPTにさせているので、不足があったり間違った形式の出力になることも多いです。上からチェックしていって、間違っているセルとそれ以降のセルを、再実行してください。

（管理メモ）インポートを試す：
https://md2.kumamoto-u.ac.jp/question/edit.php?courseid=2

In [1]:
# openai python ライブラリーをインストールしていない場合はこれを実行してください。
! pip install openai

In [2]:
import openai
openai.api_key = input("openai API key:")


In [3]:
# gpt-3.5-turbo だと、入力＋出力で4096トークン（日本語で4000文字〜5000文字）まで
# 以下のようなMoodle小テスト問題生成だと４問くらいが限界
# model ="gpt-3.5-turbo"
# max_tokens=2000

# gpt-3.5-turbo-16k だと、入力＋出力で16000トークン（日本語で1.6万文字くらい）まで
# 利用料は２倍高い https://openai.com/pricing
model="gpt-3.5-turbo-16k"
max_tokens=8000

# model="gpt-4"
# max_tokens=4000

openai.ChatCompletion.createメソッドのパラメータ：
- model (必須): 使用するモデルのIDを指定します。例えば、"gpt-3.5-turbo"です。
- messages (必須): チャット対話のリストを含むオブジェクトです。各メッセージはrole（"system"、"user"、または"assistant"）とcontent（メッセージのテキスト）を含む辞書として表されます。対話の開始時には、システムメッセージが推奨されます。
- max_tokens (オプション): 応答の最大トークン数を制限します。
- temperature (オプション): モデルの応答の多様性を制御します。値が低いほど、より決定論的な応答が生成されます（0.0に近づくほど決定的になります）、高いほどランダムな応答が生成されます（1.0に近づくほどランダムになります）。
- n (オプション): 応答の候補として生成するトークンの数を制御します。値を増やすと、より多くの応答が生成されますが、処理時間が増える可能性があります。
- stop (オプション): モデルの生成を停止するためのトリガーテキストを指定します。
- logprobs (オプション): 応答のトークンごとの対数確率を返すかどうかを制御します。

In [4]:
import datetime
now = datetime.datetime.now()
print("開始時刻 " + str(now))

開始時刻 2023-08-27 01:24:59.662141


In [5]:
# 出題したい内容を説明する文章を書く

text1 = '''
CA社とは一体何なのか？

まず、CA社がどのような企業だったのかを振り返ってみる。

CA社は2013年12月に米国で設立された企業で、英国の企業であるSCLエレクションズ社が所有している。SCLエレクションズのSCLとは「Storategy Communication Laboratories（戦略的コミュニケーション研究所」のことを指す。ニックスは2003年からSCLに参加しており、2010年にはSCLの主導権を握り、選挙コンサルティング事業を手掛けるようになる。

CA社は建前では、有権者のプロファイリング及びマーケティングを提供するデータ分析・コンサルティング会社ということになっていた。ただ、選挙コンサルティングを手掛けるSCLエレクションズの子会社であることを考慮すると、CA社はプロパガンダ企業だという見方もできる。これについてはワイリーも、「CA社はプロパガンダ企業だ」と述べている。

そんなCA社とSCLエレクションズ社は、2013年の暮れから2014年初めの時期に、ケンブリッジ大学計量心理学センターの研究に注目していた。その研究は、Facebookの個人情報、特に「いいね」を利用することで、個人の性格特性を予測できるというものだった。

そしてSCLエレクションズ社の代表者が、同センターに接触し、研究者の一人であるアレクサンドル・コーガンと協力することになる。コーガンは、Facebookと共同研究していた経験があり、Facebookの利用者及び「友だち」のプロフィールを収集できるアプリケーションに登録していた。ちなみに、このように友だちのデータを取り込む機能は「友だちAPI」と呼ばれている。カイザーによれば、2012年のオバマキャンペーンの際にも「友だちAPI」が利用されていたようだ。

そして2014年5月に、コーガンはグローバル・サイエンス・リサーチ（GSR）社を設立する。同年6月には、GSR社とSCLエレクションズ社で合意が結ばれ、Facebookに登録していたアプリケーションを改修し「GSRApp」と名付けた。

GSR社とSCLエレクションズ社はGSRAppを用いて、ユーザーデータの収集に努めていたが、2015年5月にFacebookが「友だちAPI」によるデータ収集を禁止したことで、GSRAppのデータ収集は終了した。

そして2015年12月、FacebookはコーガンとSCLエレクションズ社に対して、保有しているFacebookデータを全て削除するように要求し、コーガン氏とSCLエレクションズ社はそれを保証した。しかし、他の関係者（具体的にはCA社）がFacebookデータを所有していたことが後に判明する。そしてCA社は、このFacebookデータを利用して、選挙コンサルティングを実施することになる。
'''
# source  https://acompany.tech/privacytechlab/cambridge-analytica-and-facebook-brexit-privacy-protect/

print("# 改行やタイトルも含む文字数: %d " % len(text1))

# 改行やタイトルも含む文字数: 1190 


In [6]:
prstr1 = "以下の文章の要約文を300字程度の日本語で作成し、「要約文」というタイトルを付けて出力してください。\n\n" 
prstr1 += text1

import time
now0 = time.time()
response = openai.ChatCompletion.create(
    model = model,
    max_tokens = max_tokens,
    messages=[
        {"role": "user", "content": prstr1},
    ],
)
print("# %f [sec] " % (time.time()-now0))

res1 = response.choices[0]["message"]["content"].strip()
print("# 改行やタイトルも含む文字数: %d " % len(res1))
print(res1)

# 13.682135 [sec] 
# 改行やタイトルも含む文字数: 376 
要約文:
「CA社」とは、2013年に米国で設立された企業であり、英国のSCLエレクションズ社の子会社である。SCLエレクションズ社は選挙コンサルティング事業を手掛ける会社であり、CA社は有権者のプロファイリング及びマーケティングを提供するデータ分析・コンサルティング会社とされている。しかし、実際には「プロパガンダ企業」であるとも見なされている。2014年には、ケンブリッジ大学の研究を利用してFacebookの個人情報を収集し、2015年にはFacebookのデータ収集が禁止されるまで、ユーザーデータの収集を行っていた。しかし、Facebookはコーガン氏とSCLエレクションズ社に対してデータ削除を要求したが、別の関係者であるCA社がデータを所有していたことが判明している。そして、CA社はこのデータを利用して選挙コンサルティングを行っていた。


In [7]:
prstr2 = '''
先ほど出力された「要約文」の中の重要なキーワードを4個から8個程度、列挙し、「キーワードリスト」というタイトルを付けて出力してください。
ただし、人名や企業名はなるべく対象にしないでください。
'''

now0 = time.time()
response = openai.ChatCompletion.create(
    model = model,
    max_tokens = max_tokens,
    messages=[
        {"role": "user", "content": prstr1},
        {"role": "assistant", "content": res1},
        {"role": "user", "content": prstr2},
    ],
)
print("# %f [sec] " % (time.time()-now0))

res2 = response.choices[0]["message"]["content"].strip()
print(res2)

# 5.003763 [sec] 
キーワードリスト:
- CA社
- SCLエレクションズ社
- プロパガンダ企業
- 有権者のプロファイリング
- データ分析・コンサルティング会社
- Facebook
- 個人情報の収集
- ユーザーデータ
- 選挙コンサルティング
- データ削除


In [8]:
prstr3 = '''
これまで出力した「要約文」を使って、穴埋め問題（Moodle用）を作ってください。
「キーワードリスト」の中の各キーワードに相当する箇所を空欄にする形で作成してください。
ただし、空欄は [[1]] [[2]] [[3]] のように示してください。
また、それぞれの空欄に対応する正解を示し、それぞれの正解に類似したダミーのキーワードも書き添えてください。
'''

now0 = time.time()
response = openai.ChatCompletion.create(
    model = model,
    max_tokens = max_tokens,
    messages=[
        {"role": "user", "content": prstr1},
        {"role": "assistant", "content": res1},
        {"role": "user", "content": prstr2},
        {"role": "assistant", "content": res2},
        {"role": "user", "content": prstr3},
    ],
)
print("# %f [sec] " % (time.time()-now0))
res3 = response.choices[0]["message"]["content"].strip()
print(res3)

# 12.704377 [sec] 
以下の穴埋め問題を作成しました。正解としてのキーワードとダミーのキーワードも併せて示しています。

「CA社」とは、2013年に米国で設立された[[1]]であり、英国の[[2]]の子会社である。SCLエレクションズ社は[[3]]を手掛ける会社であり、CA社は有権者のプロファイリング及びマーケティングを提供する[[4]]とされている。しかし、実際には「[[5]]」であるとも見なされている。

正解：
[[1]]企業
[[2]]SCLエレクションズ社
[[3]]選挙コンサルティング事業
[[4]]データ分析・コンサルティング会社
[[5]]プロパガンダ企業

ダミーのキーワード：
- [[1]]学校
- [[2]]ホテル
- [[3]]販売店
- [[4]]マーケティング会社
- [[5]]石油会社


In [9]:
prstr4 = '''
先ほど出力した問題を、文章はそのまま変更せずに Drag and drop into text タイプの Moodle XML 形式に変換してください。
答えの候補として、ダミーのキーワードも含めてください。
Drag and drop into text タイプの Moodle XML 形式は、例えば以下のようなものです。 

<quiz>
  <question type="ddwtos">
    <name>
      <text>問題名</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[ 〇〇では、[[1]]の経緯と[[2]]について、[[3]]と[[4]]が述べられ、[[5]]です。]]></text>
    </questiontext>
    <generalfeedback format="html">
      <text></text>
    </generalfeedback>
    <defaultgrade>1</defaultgrade>
    <shuffleanswers>0</shuffleanswers>
    <shownumcorrect/>
    <dragbox>
      <text>正解1</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>正解2</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>正解3</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>正解4</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>正解5</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>ダミーのキーワード1</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>ダミーのキーワード2</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>ダミーのキーワード3</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>ダミーのキーワード4</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>ダミーのキーワード5</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    </question>
</quiz>
'''

now0 = time.time()
response = openai.ChatCompletion.create(
    model = model,
    max_tokens = max_tokens,
    messages=[
        {"role": "user", "content": prstr3},
        {"role": "assistant", "content": res3},
        {"role": "user", "content": prstr4},
    ],
)
print("# %f [sec] " % (time.time()-now0))
res4 = response.choices[0]["message"]["content"].strip()
print(res4)

with open('moodle-quiz-questions02.xml', 'w') as f:
    print(res4, file=f)

# 24.843309 [sec] 
以下が、先ほど作成した問題のMoodle XML形式のコードです。

```xml
<quiz>
  <question type="ddwtos">
    <name>
      <text>穴埋め問題</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[「CA社」とは、2013年に米国で設立された[[1]]であり、英国の[[2]]の子会社である。SCLエレクションズ社は[[3]]を手掛ける会社であり、CA社は有権者のプロファイリング及びマーケティングを提供する[[4]]とされている。しかし、実際には「[[5]]」であるとも見なされている。]]></text>
    </questiontext>
    <generalfeedback format="html">
      <text></text>
    </generalfeedback>
    <defaultgrade>1</defaultgrade>
    <shuffleanswers>0</shuffleanswers>
    <shownumcorrect/>
    <dragbox>
      <text>企業</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>SCLエレクションズ社</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>選挙コンサルティング事業</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>データ分析・コンサルティング会社</text>
      <group>1</group>
      <infinite/>
    </dragbox>
    <dragbox>
      <text>プロパガンダ企業</text>
      <gr

In [10]:
now2 = datetime.datetime.now()
print("終了時刻 " + str(now2))
print("所要時間 " + str(now2-now))

終了時刻 2023-08-27 01:25:55.965078
所要時間 0:00:56.302937
